In [1]:
import random
from numpy.random import seed
seed(3)
import numpy as np
np.random.default_rng
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import read_csv
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from os.path import expanduser as ospath
import time
import os
os.chdir("C:/Users/rober/project/B_Data_pre_processing/My_functions")
from B_pre_processing_data import pre_process_db
from A_merge_datasets import merge_select_columns
os.chdir("C:/Users/rober/project/F_Extra_Experiments_Abstracts/")


import xgboost as xgb
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.model_selection import KFold,GridSearchCV, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter

from gensim.models import Word2Vec
import gensim

# Steps:

>1. Load Pickley files
>2. Load unlabelled dataset
>3. Delete the patents from the unlabelled dataset that are solo in the labelled dataset
>4. Labelling the unlabelled dataset using the predition model saved when trained the best model for the abstarcts
>5. Train multple learning algorithms to choose one as the best
>6. Train the best model - Random Forest
>7. Train the best model - XGBoost
>8. Evalute the best model with the labeleld test data - manually labelled data

## 1. Load Pickle files
I load the **test features and test targets** that I used to learn the Binary classification model saved in **'project\E_Models\Abstracts'**. I need the test dataset to evaluate the new learning model which I train with data with pseudo labels with data manually labelled data. 

In [34]:
with open(ospath('~/code_final_project/E_Models/Abstracts/Pickle_files/X_test_abstracts.pickle'), 'rb') as data:
    X_test = pickle.load(data)
    
with open(ospath('~/code_final_project/E_Models/Abstracts/Pickle_files/y_test_abstracts.pickle'), 'rb') as data:
    y_test = pickle.load(data)
    
with open(ospath('~/code_final_project/E_Models/Abstracts/Pickle_files/X_train_abstracts.pickle'), 'rb') as data:
    X_train = pickle.load(data)
    
with open(ospath('~/code_final_project/E_Models/Abstracts/Pickle_files/y_train_abstracts.pickle'), 'rb') as data:
    y_train = pickle.load(data)
    
with open(ospath('~/code_final_project/B_Data_pre_processing/Pickle_files/Abstracts_cleaned.pickle'), 'rb') as data:
    db = pickle.load(data)
    
with open(ospath('~/code_final_project/E_Models/Abstracts/Pickle_files/xgb_model_abstracts.pickle'), 'rb') as data:
    xgb_fitted = pickle.load(data)
    
    
with open(ospath('~/code_final_project/E_Models/Abstracts/Pickle_files/tfidf_tune_abstracts.pickle'), 'rb') as data:
    tf_fitted = pickle.load(data)

## 2. Load the unlabelled datasets
Load all the datasets collected form USPTO and EPO offices which are unlabelled
Data collected using the scripts in **'project\A_Data_Collection'** and stored in **'project/Unlabelled_data'**.

**USPTO** patents

In [3]:
#USPTO datasets unlabelled
dt_LIB_us = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/DB_LIB_USPTO.xlsx'),dtype=str)
dt_LC_us = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/DB_LC_USPTO.xlsx'),dtype=str)
dt_SC_us = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/DB_SC_USPTO.xlsx'),dtype=str)
dt_FW_us = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/DB_FW_USPTO.xlsx'),dtype=str)

#drops rows with NAs
dt_LIB_us = dt_LIB_us[dt_LIB_us['Publn_Nr'].notna()]
dt_LC_us = dt_LC_us[dt_LC_us['Publn_Nr'].notna()]
dt_SC_us = dt_SC_us[dt_SC_us['Publn_Nr'].notna()]
dt_FW_us = dt_FW_us[dt_FW_us['Publn_Nr'].notna()]


#keeps only three columns
dt_FW_us = dt_FW_us[['Publn_Nr', 'Type', 'Text']]
dt_LIB_us = dt_LIB_us[['Publn_Nr', 'Type', 'Text']]
dt_LC_us = dt_LC_us[['Publn_Nr', 'Type', 'Text']]
dt_SC_us = dt_SC_us[['Publn_Nr', 'Type', 'Text']]

#change to lowercase name columns
dt_FW_us.columns = ['publn_nr', 'type', 'text']
dt_LIB_us.columns = ['publn_nr', 'type', 'text']
dt_LC_us.columns = ['publn_nr', 'type', 'text']
dt_SC_us.columns = ['publn_nr', 'type', 'text']

#converts the column Publn_Nr is int type
dt_LIB_us['publn_nr'] = np.int64(dt_LIB_us['publn_nr'])
dt_LC_us['publn_nr'] = np.int64(dt_LC_us['publn_nr'])
dt_FW_us['publn_nr'] = np.int64(dt_FW_us['publn_nr'])
dt_SC_us['publn_nr'] = np.int64(dt_SC_us['publn_nr'])

**EPO** patents.

In [4]:
#EPO datasets unlabelled
dt_LIB_epo = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/LIB_EPO.xlsx'),dtype=str)
dt_LC_epo = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/LC_EPO.xlsx'),dtype=str)
dt_SC_epo = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/SC_EPO.xlsx'),dtype=str)
dt_FW_epo = pd.read_excel(ospath('~/code_final_project/Unlabelled_data/FW_EPO.xlsx'),dtype=str)

#drops rows with NAs
dt_LIB_epo = dt_LIB_epo[dt_LIB_epo['Text'].notna()]
dt_LC_epo = dt_LC_epo[dt_LC_epo['Text'].notna()]
dt_SC_epo = dt_SC_epo[dt_SC_epo['Text'].notna()]
dt_FW_epo = dt_FW_epo[dt_FW_epo['Text'].notna()]

#converts the column Publn_Nr is int type
dt_LIB_epo['Publn_Nr'] = np.int64(dt_LIB_epo['Publn_Nr'])
dt_LC_epo['Publn_Nr'] = np.int64(dt_LC_epo['Publn_Nr'])
dt_FW_epo['Publn_Nr'] = np.int64(dt_FW_epo['Publn_Nr'])
dt_SC_epo['Publn_Nr'] = np.int64(dt_SC_epo['Publn_Nr'])

#keeps only three columns
dt_FW_epo = dt_FW_epo[['Publn_Nr', 'Type', 'Text']]
dt_LIB_epo = dt_LIB_epo[['Publn_Nr', 'Type', 'Text']]
dt_LC_epo = dt_LC_epo[['Publn_Nr', 'Type', 'Text']]
dt_SC_epo = dt_SC_epo[['Publn_Nr', 'Type', 'Text']]

#change to lowercase name columns
dt_FW_epo.columns = ['publn_nr', 'type', 'text']
dt_LIB_epo.columns = ['publn_nr', 'type', 'text']
dt_LC_epo.columns = ['publn_nr', 'type', 'text']
dt_SC_epo.columns = ['publn_nr', 'type', 'text']

Merge in one dataset.

In [5]:
unlab_data = merge_select_columns(dt_LIB_us, dt_SC_us, dt_FW_us, dt_LC_us, dt_LIB_epo, dt_LC_epo, dt_FW_epo, dt_SC_epo, labels = 'No')

In [6]:
unlab_data

,publn_nr,type,text
0,10354828,Abstract,According to an embodiment of the present disc...
11,10085792,Abstract,An apparatus includes an instrument body and t...
31,9782215,Abstract,An ultrasonic surgical device comprises a hand...
52,10343141,Abstract,MOFs are disclosed that can efficiently adsorb...
60,10337115,Abstract,Surface treated copper foils for use in high s...
...,...,...,...
57635,3322002,Abstract,The present invention relates to an electroly...
57650,3327827,Abstract,The present invention provides a battery cell...
57682,3336962,Abstract,"Methods of preparing hetero ionic complexes, ..."
57707,3349273,Abstract,The present disclosure relates to an electrod...


## 3. Delete the patents from the unlabelled dataset that are also in the labelled dataset

In [7]:
#save in list the publication numbers of the patents used for the logistic above
list_publ_num_uspto = db['publn_nr'].unique()

#keeps only patents that are not used to train and test the best model for the Abstracts saved
unlab_data = unlab_data[~unlab_data['publn_nr'].isin(list_publ_num_uspto)]
print(len(list_publ_num_uspto))

194


Pre-process the text

In [8]:
unlab_data = pre_process_db(unlab_data)#pre-process the text

In [9]:
unlab_data.head()

,publn_nr,type,text,text_clean
0,10354828,Abstract,According to an embodiment of the present disc...,accord embodiment present disclosure photocath...
11,10085792,Abstract,An apparatus includes an instrument body and t...,apparatus include instrument body transmission...
31,9782215,Abstract,An ultrasonic surgical device comprises a hand...,ultrasonic surgical device comprise handle ass...
52,10343141,Abstract,MOFs are disclosed that can efficiently adsorb...,mofs disclose efficiently adsorb oxygen gas st...
60,10337115,Abstract,Surface treated copper foils for use in high s...,surface treat copper foil use high speed circu...


## 4. Labelling the unlabelled dataset using the predition model saved when trained the best model for the abstarcts

With the best model saved from the binary classification model of the abstracts - **project\E_Models\Abstracts** - and with the TF-IDF vectorizer also for the same best model, I predict the labels for the unlabelled dataset.

In [35]:
#word representation using the TF-IDF fitted for the best binary model of the abstracts
X = unlab_data['text_clean']
X_tf = tf_fitted.transform(X).toarray()

In [36]:
pred_lab = xgb_fitted.predict(X_tf)
unlab_data['pred_labels'] = pred_lab

In [37]:
unlab_data

,publn_nr,type,text,text_clean,pred_labels
0,10354828,Abstract,According to an embodiment of the present disc...,accord embodiment present disclosure photocath...,1
11,10085792,Abstract,An apparatus includes an instrument body and t...,apparatus include instrument body transmission...,1
31,9782215,Abstract,An ultrasonic surgical device comprises a hand...,ultrasonic surgical device comprise handle ass...,1
52,10343141,Abstract,MOFs are disclosed that can efficiently adsorb...,mofs disclose efficiently adsorb oxygen gas st...,1
60,10337115,Abstract,Surface treated copper foils for use in high s...,surface treat copper foil use high speed circu...,1
...,...,...,...,...,...
57635,3322002,Abstract,The present invention relates to an electroly...,present invention relate electrolyte impregnat...,1
57650,3327827,Abstract,The present invention provides a battery cell...,present invention provide battery cell include...,1
57682,3336962,Abstract,"Methods of preparing hetero ionic complexes, ...",methods prepare hetero ionic complexes ionic l...,0
57707,3349273,Abstract,The present disclosure relates to an electrod...,present disclosure relate electrode manufactur...,1


In [38]:
unlab_data.groupby(by='pred_labels').count()

,publn_nr,type,text,text_clean
pred_labels,,,,
0,3069,3069,3069,3069
1,105422,105422,105422,105422


Extremely unbalenced

## 5. train multple learning algorithms to choose one as the best

I run three algorithm, a Logistic Regression, XGBoost and Random Forest

In [39]:
X_train_pseudo = unlab_data['text_clean']
y_train_pseudo = unlab_data['pred_labels']
X_train_pseudod_tf = tf_fitted.transform(X_train_pseudo).toarray()

In [40]:
X_test_tf = tf_fitted.transform(X_test).toarray()

In [41]:
oversample = SMOTE()
X_train_pseudo_bal, y_train_pseudo_bal = oversample.fit_resample(X_train_pseudod_tf, y_train_pseudo)

### XGBoost

In [43]:
seed(3)
xbg = xgb.XGBClassifier(objective = 'binary:logistic', n_jobs=-1)
xbg.fit(X_train_pseudo_bal, y_train_pseudo_bal )
xbg_pred = xbg.predict(X_test_tf)

print("Accuracy on the test data: ")
print(metrics.accuracy_score(y_test, xbg_pred))
print("Classification report")
print(metrics.classification_report(y_test, xbg_pred))
print('Confusion Matrix : ')
print(metrics.confusion_matrix(y_test, xbg_pred))

Accuracy on the test data: 
0.9102564102564102
Classification report
              precision    recall  f1-score   support

           0       0.50      0.14      0.22         7
           1       0.92      0.99      0.95        71

    accuracy                           0.91        78
   macro avg       0.71      0.56      0.59        78
weighted avg       0.88      0.91      0.89        78

Confusion Matrix : 
[[ 1  6]
 [ 1 70]]


### Logistic Regression

In [18]:
logi = LogisticRegression( random_state=0, n_jobs=-1)
logi.fit(X_train_pseudo_bal, y_train_pseudo_bal)
logi_pred = logi.predict(X_test_tf)


print("Accuracy on the test data: ")
print(metrics.accuracy_score(y_test, logi_pred))
print("Classification report")
print(metrics.classification_report(y_test, logi_pred))
print('Confusion Matrix : ')
print(metrics.confusion_matrix(y_test, logi_pred))

Accuracy on the test data: 
0.8333333333333334
Classification report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.90      0.92      0.91        71

    accuracy                           0.83        78
   macro avg       0.45      0.46      0.45        78
weighted avg       0.82      0.83      0.83        78

Confusion Matrix : 
[[ 0  7]
 [ 6 65]]


### Random Forest

In [45]:
random.seed(2)
rf = RandomForestClassifier(random_state = 8, n_jobs = -1)
rf.fit(X_train_pseudo_bal, y_train_pseudo_bal)
rf_pred = rf.predict(X_test_tf)

print("Accuracy on validation data: \n",metrics.accuracy_score(y_test, rf_pred))
print("Classification report: \n",metrics.classification_report(y_test, rf_pred))
print('Confusion Matrix validation: \n' + str(metrics.confusion_matrix(y_test, rf_pred)))

Accuracy on validation data: 
 0.9230769230769231
Classification report: 
               precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.92      1.00      0.96        71

    accuracy                           0.92        78
   macro avg       0.96      0.57      0.60        78
weighted avg       0.93      0.92      0.90        78

Confusion Matrix validation: 
[[ 1  6]
 [ 0 71]]


## 6. Train the best model - Random Forest

I will run and tune the random forest algorithm since it is the best one here

### I split first the unlabelled dataset with the predicted labels in three sets. 
### Train 50%, validation 25% and test 25%.

In [48]:
X_train_ps, X_val_ps, y_train_ps , y_val_ps = train_test_split(unlab_data['text_clean'], 
                                                    unlab_data['pred_labels'], test_size=0.5, random_state=8, stratify = unlab_data['pred_labels'])

X_val_ps, X_test_ps, y_val_ps , y_test_ps = train_test_split(X_val_ps, 
                                                    y_val_ps, test_size=0.5, random_state=8, stratify = y_val_ps)


### Tune Hyperparameters TF-IDF

In [49]:
random.seed(2)
pipe_grid = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase=False), ),
    ('clf', OneVsRestClassifier(RandomForestClassifier(random_state = 8))),
])
param_grid = {
    'tfidf__norm' :('l1', 'l2'),
    'tfidf__max_features' : (300, 350, 400, 450,500),
    'tfidf__sublinear_tf' : [True, False]
}

grid_search_tfidf = GridSearchCV(pipe_grid, param_grid, cv=2, n_jobs=2, verbose=3)
grid_search_tfidf.fit(X_val_ps, y_val_ps)


print("Best set of parameters: \n" ,grid_search_tfidf.best_estimator_.steps)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   52.5s
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:  1.3min finished


Best set of parameters: 
 [('tfidf', TfidfVectorizer(lowercase=False, max_features=300, norm='l1')), ('clf', OneVsRestClassifier(estimator=RandomForestClassifier(random_state=8)))]


In [50]:
tfidf_tune = grid_search_tfidf.best_estimator_[0]

X_train_ps_tf_tune = tfidf_tune.fit_transform(X_train_ps).toarray()
X_test_ps_tf_tune = tfidf_tune.transform(X_test_ps).toarray()
X_val_ps_tf_tune = tfidf_tune.transform(X_val_ps).toarray()

### By using SMOTE, I balanced the unlabelled dataset since it also very unbalanced.

In [51]:
balance = SMOTE()
X_train_ps_tf_tune_bal, y_train_ps_tf_tune_bal = balance.fit_resample(X_train_ps_tf_tune, y_train_ps)

### Train and evaluate Random Forest algorithm with TF-IDF tuned

In [52]:
random.seed(2)
rf_tf_tune = RandomForestClassifier(random_state = 8, n_jobs = -1)
rf_tf_tune.fit(X_train_ps_tf_tune_bal, y_train_ps_tf_tune_bal)
rf_tf_tune_pred = rf_tf_tune.predict(X_val_ps_tf_tune)

print("Accuracy on validation data: \n",metrics.accuracy_score(y_val_ps, rf_tf_tune_pred))
print("Classification report: \n",metrics.classification_report(y_val_ps, rf_tf_tune_pred))
print('Confusion Matrix validation: \n' + str(metrics.confusion_matrix(y_val_ps, rf_tf_tune_pred)))


Accuracy on validation data: 
 0.9929211370423626
Classification report: 
               precision    recall  f1-score   support

           0       0.96      0.78      0.86       768
           1       0.99      1.00      1.00     26355

    accuracy                           0.99     27123
   macro avg       0.98      0.89      0.93     27123
weighted avg       0.99      0.99      0.99     27123

Confusion Matrix validation: 
[[  598   170]
 [   22 26333]]


In [53]:
random.seed(2)
rf_tf_tune_pred_test = rf_tf_tune.predict(X_test_ps_tf_tune)
print("Accuracy on the test data: \n", metrics.accuracy_score(y_test_ps, rf_tf_tune_pred_test))
print("Classification report: \n",metrics.classification_report(y_test_ps, rf_tf_tune_pred_test))
print('Confusion Matrix test: \n' + str(metrics.confusion_matrix(y_test_ps, rf_tf_tune_pred_test)))

Accuracy on the test data: 
 0.9928842679644582
Classification report: 
               precision    recall  f1-score   support

           0       0.98      0.77      0.86       767
           1       0.99      1.00      1.00     26356

    accuracy                           0.99     27123
   macro avg       0.98      0.88      0.93     27123
weighted avg       0.99      0.99      0.99     27123

Confusion Matrix test: 
[[  588   179]
 [   14 26342]]


### Tune hyperparameters Random Forest algorithm

In [55]:
param_grid = {'min_samples_leaf': [1, 2, 4],
              'min_samples_split' : [2, 5, 10],
             'max_features' : ['auto', 'sqrt'],
              'max_depth': [20, 40, 60, 80, 100, None],
              'n_estimators' : [200, 400, 600, 800, 1000],
             'bootstrap' : [True, False]}

In [56]:
random.seed(2)
rand_for = RandomForestClassifier(random_state=8)

random_search_rf = RandomizedSearchCV(estimator=rand_for,
                                   param_distributions=param_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1)

# Fit the random search model
random_search_rf.fit(X_val_ps_tf_tune, y_val_ps)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 82.0min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=8),
                   n_iter=50,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [20, 40, 60, 80, 100,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000]},
                   scoring='accuracy', verbose=1)

### Train and evaluate Random forest with both TF-IDF and algorithm tuned

In [76]:
random.seed(2)
rf_both_tune = random_search_rf.best_estimator_
rf_both_tune.fit(X_train_ps_tf_tune_bal, y_train_ps_tf_tune_bal)
rf_both_tune_pred = rf_both_tune.predict(X_val_ps_tf_tune)

print("Accuracy on validation data: \n",metrics.accuracy_score(y_val_ps, rf_both_tune_pred))
print("Classification report: \n",metrics.classification_report(y_val_ps, rf_both_tune_pred))
print('Confusion Matrix validation: \n' + str(metrics.confusion_matrix(y_val_ps, rf_both_tune_pred)))

Accuracy on validation data: 
 0.9929211370423626
Classification report: 
               precision    recall  f1-score   support

           0       0.98      0.77      0.86       768
           1       0.99      1.00      1.00     26355

    accuracy                           0.99     27123
   macro avg       0.98      0.88      0.93     27123
weighted avg       0.99      0.99      0.99     27123

Confusion Matrix validation: 
[[  591   177]
 [   15 26340]]


In [59]:
random.seed(2)
rf_both_tune_pred_test = rf_both_tune.predict(X_test_ps_tf_tune)
print("Accuracy on the test data: \n", metrics.accuracy_score(y_test_ps, rf_both_tune_pred_test))
print("Classification report: \n",metrics.classification_report(y_test_ps, rf_both_tune_pred_test))
print('Confusion Matrix test: \n' + str(metrics.confusion_matrix(y_test_ps, rf_both_tune_pred_test)))

Accuracy on the test data: 
 0.9923681008737971
Classification report: 
               precision    recall  f1-score   support

           0       0.98      0.75      0.85       767
           1       0.99      1.00      1.00     26356

    accuracy                           0.99     27123
   macro avg       0.99      0.87      0.92     27123
weighted avg       0.99      0.99      0.99     27123

Confusion Matrix test: 
[[  573   194]
 [   13 26343]]


### Evaluate the model using the test data set split from the labelled dataset
Random Forest with only TF-IDF tuned

In [77]:
X_test_tf_tune = tfidf_tune.transform(X_test).toarray()

In [78]:
random.seed(2)
rf_tf_tune_pred_test_labelled = rf_tf_tune.predict(X_test_tf_tune)
print("Accuracy on the test data: \n", metrics.accuracy_score(y_test, rf_tf_tune_pred_test_labelled))
print("Classification report: \n",metrics.classification_report(y_test, rf_tf_tune_pred_test_labelled))
print('Confusion Matrix test: \n' + str(metrics.confusion_matrix(y_test, rf_tf_tune_pred_test_labelled)))

Accuracy on the test data: 
 0.9230769230769231
Classification report: 
               precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.92      1.00      0.96        71

    accuracy                           0.92        78
   macro avg       0.96      0.57      0.60        78
weighted avg       0.93      0.92      0.90        78

Confusion Matrix test: 
[[ 1  6]
 [ 0 71]]


## 7. Train the best model - XGBoost

### Tune Hyperparameters for TF-IDF for XGBoost

In [60]:
random.seed(2)
pipeline_grid = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase=False)),
    ('clf', OneVsRestClassifier(xgb.XGBClassifier(objective = 'binary:logistic', n_jobs=-1))),
])
param_grid = {
    'tfidf__norm' :('l1', 'l2'),
    'tfidf__max_features' : (300, 350, 400, 450,500),
    'tfidf__sublinear_tf' : [True, False]

}

grid_search_tfidf_xgb = GridSearchCV(pipeline_grid, param_grid, cv=2, verbose=1, n_jobs=-1)
grid_search_tfidf_xgb.fit(X_val_ps, y_val_ps)

print("Best parameters for the TF-IDF: {}".format(grid_search_tfidf_xgb.best_estimator_.steps))

Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   52.6s finished


Best parameters for the TF-IDF: [('tfidf', TfidfVectorizer(lowercase=False, max_features=400, norm='l1', sublinear_tf=True)), ('clf', OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=-1,
                                            num_paral

### Fit with the new vectorizer tune and balance the data with SMOTE

In [61]:
tfidf_xb = grid_search_tfidf_xgb.best_estimator_[0]

X_train_ps_tf_tune_xb = tfidf_xb.fit_transform(X_train_ps).toarray()
X_test_ps_tf_tune_xb = tfidf_xb.transform(X_test_ps).toarray()
X_val_ps_tf_tune_xb = tfidf_xb.transform(X_val_ps).toarray()

In [62]:
balance = SMOTE()
X_train_ps_tf_tune_bal_xb, y_train_ps_tf_tune_bal_xb = balance.fit_resample(X_train_ps_tf_tune_xb, y_train_ps)

### Train and evaluate the model XGBoost with TF-IDF tuned 

In [63]:
random.seed(2)
xb_tf_tune = xgb.XGBClassifier(objective = 'binary:logistic', n_jobs=-1)
xb_tf_tune.fit(X_train_ps_tf_tune_bal_xb, y_train_ps_tf_tune_bal_xb)
xb_tf_tune_pred = xb_tf_tune.predict(X_val_ps_tf_tune_xb)


print("Accuracy on validation data: \n",metrics.accuracy_score(y_val_ps, xb_tf_tune_pred))
print("Classification report: \n",metrics.classification_report(y_val_ps, xb_tf_tune_pred))
print('Confusion Matrix validation: \n' + str(metrics.confusion_matrix(y_val_ps, xb_tf_tune_pred)))

Accuracy on validation data: 
 0.9966080448327987
Classification report: 
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       768
           1       1.00      1.00      1.00     26355

    accuracy                           1.00     27123
   macro avg       0.97      0.97      0.97     27123
weighted avg       1.00      1.00      1.00     27123

Confusion Matrix validation: 
[[  724    44]
 [   48 26307]]


In [64]:
random.seed(2)
xb_tf_tune_pred_test = xb_tf_tune.predict(X_test_ps_tf_tune_xb)
print("Accuracy on test data: \n",metrics.accuracy_score(y_test_ps, xb_tf_tune_pred_test))
print("Classification report: \n",metrics.classification_report(y_test_ps, xb_tf_tune_pred_test))
print('Confusion Matrix test: \n' + str(metrics.confusion_matrix(y_test_ps, xb_tf_tune_pred_test)))

Accuracy on test data: 
 0.9963868303653726
Classification report: 
               precision    recall  f1-score   support

           0       0.94      0.93      0.94       767
           1       1.00      1.00      1.00     26356

    accuracy                           1.00     27123
   macro avg       0.97      0.97      0.97     27123
weighted avg       1.00      1.00      1.00     27123

Confusion Matrix test: 
[[  715    52]
 [   46 26310]]


### Tune Hyperparameters XGBoost

In [65]:
param_grid = {
    'max_depth': [2,4,6,8,10],
    'n_estimators': [60, 10,120,160,200, 240],
    'learning_rate': [0.1, 0.01, 0.05, 0.009]
}

In [67]:
random.seed(2)
xb = xgb.XGBClassifier(objective = 'binary:logistic', n_jobs=-1)



# Definition of the random search
random_search_xb = RandomizedSearchCV(estimator=xb,
                                   param_distributions=param_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1)

# Fit the random search model
random_search_xb.fit(X_val_ps_tf_tune_xb, y_val_ps)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 23.2min finished


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=-1,
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                            

In [68]:
random.seed(2)
xb_both_tune = random_search_xb.best_estimator_
xb_both_tune.fit(X_train_ps_tf_tune_bal_xb, y_train_ps_tf_tune_bal_xb)
xb_both_tune_pred = xb_both_tune.predict(X_val_ps_tf_tune_xb)


print("Accuracy on validation data: \n",metrics.accuracy_score(y_val_ps, xb_both_tune_pred))
print("Classification report: \n",metrics.classification_report(y_val_ps, xb_both_tune_pred))
print('Confusion Matrix validation: \n' + str(metrics.confusion_matrix(y_val_ps, xb_both_tune_pred)))

Accuracy on validation data: 
 0.9964974375990856
Classification report: 
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       768
           1       1.00      1.00      1.00     26355

    accuracy                           1.00     27123
   macro avg       0.97      0.97      0.97     27123
weighted avg       1.00      1.00      1.00     27123

Confusion Matrix validation: 
[[  720    48]
 [   47 26308]]


In [70]:
random.seed(2)
xb_both_tune_pred_test = xb_both_tune.predict(X_test_ps_tf_tune_xb)
print("Accuracy on test data: \n",metrics.accuracy_score(y_test_ps, xb_both_tune_pred_test))
print("Classification report: \n",metrics.classification_report(y_test_ps, xb_both_tune_pred_test))
print('Confusion Matrix test: \n' + str(metrics.confusion_matrix(y_test_ps, xb_both_tune_pred_test)))

Accuracy on test data: 
 0.9965711757548944
Classification report: 
               precision    recall  f1-score   support

           0       0.95      0.93      0.94       767
           1       1.00      1.00      1.00     26356

    accuracy                           1.00     27123
   macro avg       0.97      0.96      0.97     27123
weighted avg       1.00      1.00      1.00     27123

Confusion Matrix test: 
[[  711    56]
 [   37 26319]]


### Evaluate the model using the test data set split from the labelled dataset
XGBoost with TF-IDF and algorithm tuned

In [71]:
X_test_tf_tune = tfidf_xb.transform(X_test).toarray()

In [73]:
random.seed(2)
xb_both_tune_pred_test_original = xb_both_tune.predict(X_test_tf_tune)
print("Accuracy on test data: \n",metrics.accuracy_score(y_test, xb_both_tune_pred_test_original))
print("Classification report: \n",metrics.classification_report(y_test, xb_both_tune_pred_test_original))
print('Confusion Matrix test: \n' + str(metrics.confusion_matrix(y_test, xb_both_tune_pred_test_original)))

Accuracy on test data: 
 0.9102564102564102
Classification report: 
               precision    recall  f1-score   support

           0       0.50      0.14      0.22         7
           1       0.92      0.99      0.95        71

    accuracy                           0.91        78
   macro avg       0.71      0.56      0.59        78
weighted avg       0.88      0.91      0.89        78

Confusion Matrix test: 
[[ 1  6]
 [ 1 70]]


## Both the best model using pseudo labelling data did not improve the original model with just an abstract classified correctly out 7.